In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
df = pd.read_csv('order_brush_order.csv')

In [3]:
df['event_time'] =  pd.to_datetime(df['event_time'], infer_datetime_format=True)

In [4]:
sorted_df = df.sort_values(by=['event_time'])

In [5]:
plus1h = sorted_df['event_time'] +  pd.DateOffset(hours=1)
sorted_df['plus1h'] = plus1h

In [6]:
sorted_df.head()

,orderid,shopid,userid,event_time,plus1h
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,2019-12-27 01:00:00
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,2019-12-27 01:00:00
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,2019-12-27 01:00:00
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,2019-12-27 01:00:02
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,2019-12-27 01:00:05


In [7]:
len(sorted_df)

222750

In [8]:
## %%timeit
# for optimising, use .values to convert to numpy array, reshape is used so that we can concatenate 
event_time = np.unique(sorted_df['event_time'].values).reshape(-1,1)
plus1h = np.unique(sorted_df['plus1h'].values).reshape(-1,1)
great_sliding_window = np.concatenate((event_time,plus1h), axis = 1)

In [9]:
# From here, we observed that there are timerange where there won't be any data at all. Redundant.
great_sliding_window

array([['2019-12-27T00:00:00.000000000', '2019-12-27T01:00:00.000000000'],
       ['2019-12-27T00:00:02.000000000', '2019-12-27T01:00:02.000000000'],
       ['2019-12-27T00:00:05.000000000', '2019-12-27T01:00:05.000000000'],
       ...,
       ['2019-12-31T23:59:47.000000000', '2020-01-01T00:59:47.000000000'],
       ['2019-12-31T23:59:51.000000000', '2020-01-01T00:59:51.000000000'],
       ['2019-12-31T23:59:56.000000000', '2020-01-01T00:59:56.000000000']],
      dtype='datetime64[ns]')

In [10]:
len(great_sliding_window)

121064

In [11]:
# We now have the sliding window. We can remove orderid and plus1h from sorted.df as they are not needed
sorted_df = sorted_df.drop(['orderid','plus1h'],axis=1)
sorted_df.head()

,shopid,userid,event_time
150060,6042309,97707522,2019-12-27 00:00:00
25985,104804492,97707522,2019-12-27 00:00:00
81020,8715449,97707522,2019-12-27 00:00:00
146598,190969466,170182475,2019-12-27 00:00:02
80355,2859407,12532131,2019-12-27 00:00:05


In [12]:
# concentrate_rate = number of orders within 1 hr / number of unique buyers within 1 hr
# >= 3 : order brushing 

# a DataFrame to record which shop-user do order-brushing
order_brusher = pd.DataFrame()
tracker = 0 

for t1,t2 in great_sliding_window:
    # for every step of sliding window, retrieve all the orders that fall within that time step
    # Note: the size of temp within each time step will be different since there could be different number of orders 
    temp = sorted_df[(t1 <= sorted_df['event_time'].values) & (sorted_df['event_time'].values <= t2)]
    
    # filtering to reduce search space: for order brushing , need at least 3 orders made to the shop in that 1 hr time step
    # Commented out since transform is a major bottleneck for large scale data
    # temp = temp[temp.groupby('shopid')['shopid'].transform('size') >= 3]
    
    # no of orders to a shop in an hour
    no_orders = temp.groupby('shopid')['userid'].count() 
    
    # no of unique buyer to a shop in an hour
    no_unique_buyers = temp.groupby('shopid')['userid'].nunique()
    
    # concentration_rate 
    concentration_rate = no_orders/no_unique_buyers
    
    # retrieving the shopid where concentration_rate >= 3
    order_brushing_shopid = concentration_rate[concentration_rate>=3].index
    
    # need to find respective suspicious users in each shopid. There could be many users in cahoot with a single shop
    suspects = temp[temp['shopid'].isin(order_brushing_shopid)]

    # for each userid with respect to each shopid, we can see the count.
    # Note that it is possible for one shopid to have multiple userids having the maximum count
    suspects_counts = suspects.groupby(['shopid','userid'])['shopid'].count().reset_index(name='count')

    # max count for each category. Required this because it is possible for one shopid to have multiple userids having the maximum count
    max_counts = suspects_counts.groupby(['shopid'])['count'].agg('max').reset_index(name='count')
    
    # To retrieve a dataframe with all the top order_brushing_users 
    suspects_df = suspects_counts[suspects_counts['shopid'].isin(max_counts['shopid']) & suspects_counts['count'].isin(max_counts['count'])]
    suspects_df = suspects_df[['shopid','userid']]
    
    # Appending and dropping duplicate at each time step
    order_brusher = order_brusher.append(suspects_df)
    order_brusher = order_brusher.drop_duplicates()
    
    tracker += 1
    
    # print(tracker, order_brusher)
    

In [13]:
order_brusher

,shopid,userid
0,141025402.0,613919.0
1,181009364.0,214208720.0
0,63888299.0,112649507.0
0,9374147.0,148176353.0
2,192608876.0,213646699.0
...,...,...
1,93295101.0,39834931.0
0,9153340.0,197569856.0
0,54257623.0,107414154.0
2,189544563.0,799445.0


In [14]:
# Need to convert float to int as per submission details
order_brusher['shopid']= order_brusher['shopid'].astype(int)
order_brusher['userid']= order_brusher['userid'].astype(int)

In [15]:
# sorting is needed since all userid are to be separated by & with the smaller userid first
order_brusher.sort_values(by=['shopid','userid'])
order_brusher = order_brusher.sort_values(by=['shopid','userid'])
order_brusher

,shopid,userid
0,10159,214988798
0,10402,77819
0,10536,672345
0,42472,740844
0,42818,170385453
...,...,...
3,203531250,114282846
2,204225676,198662175
2,208696908,214111334
0,210197928,52867898


In [16]:
# serves as an intermediate checkpoint
order_brusher.to_csv('order_brusher_preprocessed.csv', index=False)

In [17]:
order_brusher['shopid']= order_brusher['shopid'].astype(str)
order_brusher['userid']= order_brusher['userid'].astype(str)
order_brusher = order_brusher.groupby('shopid')['userid'].apply('&'.join).reset_index()
order_brusher


,shopid,userid
0,100446829,2434757
1,10159,214988798
2,10199219,8405753
3,10206302,95058664
4,103715156,214226569
...,...,...
310,98481320,124597967
311,98768262,150440717
312,98793086,142710562
313,99067259,108402614


In [18]:
# For verification
order_brusher[order_brusher['shopid']=='181009364']

,shopid,userid
152,181009364,101832161&214208720


In [19]:
# Retrieving every unique shops
all_shops = pd.unique(sorted_df['shopid'])
all_shops = pd.Series(all_shops)
all_shops

0          6042309
1        104804492
2          8715449
3        190969466
4          2859407
           ...    
18765    173249173
18766    212058195
18767       577768
18768    162561288
18769    203090989
Length: 18770, dtype: int64

In [20]:
# Retrieving the order brushing shops
order_brusher['shopid']= order_brusher['shopid'].astype(int)
bad_shops = order_brusher['shopid']
bad_shops

0      100446829
1          10159
2       10199219
3       10206302
4      103715156
         ...    
310     98481320
311     98768262
312     98793086
313     99067259
314     99836251
Name: shopid, Length: 315, dtype: int32

In [21]:
all_shops.isin(bad_shops)

0        False
1        False
2         True
3        False
4        False
         ...  
18765    False
18766    False
18767    False
18768    False
18769    False
Length: 18770, dtype: bool

In [22]:
# Getting all the good shops and converting them to a Series
good_shop_series = all_shops[~all_shops.isin(bad_shops)]
good_shop_series = pd.Series(good_shop_series,name='shopid')
good_shop_series

0          6042309
1        104804492
3        190969466
4          2859407
5         94479614
           ...    
18765    173249173
18766    212058195
18767       577768
18768    162561288
18769    203090989
Name: shopid, Length: 18455, dtype: int64

In [23]:
good_shops = pd.DataFrame()

In [24]:
good_shops['shopid'] = good_shop_series
good_shops['userid'] = 0
good_shops

,shopid,userid
0,6042309,0
1,104804492,0
3,190969466,0
4,2859407,0
5,94479614,0
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [25]:
order_brusher

,shopid,userid
0,100446829,2434757
1,10159,214988798
2,10199219,8405753
3,10206302,95058664
4,103715156,214226569
...,...,...
310,98481320,124597967
311,98768262,150440717
312,98793086,142710562
313,99067259,108402614


In [26]:
result = order_brusher.append(good_shops)
result

,shopid,userid
0,100446829,2434757
1,10159,214988798
2,10199219,8405753
3,10206302,95058664
4,103715156,214226569
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [27]:
result.to_csv('result.csv', index=False)